In [29]:
import pandas as pd
import numpy as np
import time
start_time = time.time()

In [30]:
#reading csv file and storing it in a dataframe
data = pd.read_csv('simplex_data.csv')

In [31]:
# extract data from csv file and store in respective variables

m = int(data.loc[0]['m'])                         # number of rows of matrix A
n = int(data.loc[0]['n'])                         # number of column of matrix A
c = list(data.loc[0:n-1]['c'])                    # cost vector (row vector)
for i in range(m):
    c.append(0)
c = np.array(c)
#print("Cost Vector is: ",c)
b = np.array(data.loc[0:m-1]['b'])                 # b vector or the RHS vector
#print("Initial Solution vector is: ",b)

A = np.zeros((m,n))                                # decision varibales coefficient matrix

#lOOP FOR ASSIGNING ELEMENTS TO MATRIX A
k = 0
for i in range(m):
    for j in range(n):
        A[i][j] = int(data.loc[k]['A'])
        k += 1
#print("Matrix A in standard form in standard form is:\n", A)

#initial basis matrix B
B = np.identity(m)
#print("Initial Basis matrix is:\n", B)

#Adding identity matrix to the A matrix (adding initial basic feasible matrix)
A = np.c_[A,B]                                       # .c_ stacks matrix B at the right side of A
#print("Matrix A after converting into standard form is given as:\n", A)

zj_cj_list = []                                      #list to record all the values of z_j (j is all non basic variables)

In [32]:
#find inverse of basic feasible matrix as B_inv
def inverse(matrix):                        #function to find matrix inverse when argument is the matrix
    return np.linalg.inv(matrix)
B_inv = inverse(B)
#print("Intial Basis Inverse Matrix is:\n ", B_inv)

b_ = B_inv @ b                              # b_ calculates the value of basic variables at each iteration
#print("Initial solution is:\n", b_)

#finding a_j
a_i = []                                    # list that indicates what column of A are in basis
for i in range(m):
    a_i.append(n+i)                         #columns start from number 0 in python (e.g 2 indicates third column in python)
a_j = []                                    #list that reocrd what column of A are non basic
for j in range(n):
    a_j.append(j)
#print("The list of index of column of matrix A that are in basis is:\n", a_i)
#print("The list of index of column of matrix A that are non-basis is:\n", a_j)

#finding c_b 
c_b = []
for items in a_i:
    c_b.append(c[items])
c_j = []
for items in a_j:
    c_j.append(c[items])
#print("The cost vector of basis is:\n", c_b)
#print("The cost vector of non basic decision variables:\n", c_j)

#finding objective function value
z = c_b @ B_inv @ b # @ method does matrix multiplication of a matrix and a vector
#print("The initial objective function value is:\n", z)

W = c_b @ B_inv
#print("W vector is:\n", W)

In [33]:
# populating zj_cj_list
count = 0 
for items in a_j: 
    zj_cj_list.append(c_b @ B_inv @ A[:,items] - c_j[count])
    count += 1 
#print("zj_cj_list is:\n", zj_cj_list)

In [34]:
#making a list for basic and non basic variables
#x_b, x_n =[], []
#for j in range(n):
    #x_n.append(0)
#x_b = B_inv @ b - B_inv @ A[:,a_j] @ x_n  #calculating x_b
#x_b

In [35]:
Minimum = min(zj_cj_list)
while Minimum < 0:
    # finding minimum of zj - cj list 
    entering = zj_cj_list.index(Minimum) # index of entering variable
    #print(entering)

    #find the exiting variable

    y_ik, x_k_list = [], []     #check required
    y_ik = B_inv @ A[:,entering]
    #print("y is:",y_ik)

    for i in range(len(y_ik)):
        x_k_list.append(b_[i] / y_ik[i])     #minimum ratio test
    x_k = np.argmin(x_k_list) #gives the index of x_k_list where the value is minimum ()
    #print("exiting index is: ",x_k)
    temp = a_i[x_k]
    a_i[x_k] = entering
    a_j[entering] = temp

    #updating cost basis vector
    temp = c_b[x_k]
    c_b[x_k] = c_j[entering]
    c_j[entering] = temp

    # updating Basis matrix and calculating B_inv
    B = A[:,a_i]
    B_inv = inverse(B)
    
    b_ = B_inv @ b
    for i in range(len(zj_cj_list)):
        zj_cj_list[i] = (c_b @ B_inv @ A[:,a_j])[i] - c_j[i]
    Minimum = min(zj_cj_list)
    z = c_b @ b_
print ("The optimal value of given maximization objective function is: ", z)

# Assigning the decision variables values which are in the basis and all non basic variables are zero.
i = 0
for item in a_i:
    print("Basic Variable x{} =".format(item+1), b_[i])
    i += 1
print("The value of remaining decision varibales is 0")
print("The time taken to run the algorithm is: ", time.time() - start_time, "seconds")

The optimal value of given maximization objective function is:  20000.0
Basic Variable x3 = 20.0
Basic Variable x2 = 50.0
The value of remaining decision varibales is 0
The time taken to run the algorithm is:  0.9198753833770752 seconds
